In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import os
import pandas as pd


In [ ]:
N_SEGMENTS = 50

IMAGE_ROOT = "signal_img"
SIGNAL_ROOT = "signal"

In [ ]:
def gen_interpolation(x_1: float, x_2: float):
    num_interpolation = max(int((np.ceil(np.log2(np.abs(x_1-x_2))))**2), 1)
    delta = 1.0*np.abs(x_1-x_2)/num_interpolation
    interpolations = []
    for idx in range(1, num_interpolation):
        if x_1 < x_2:
            interpolations.append(x_1+idx*np.random.normal(delta, 0.1*delta))
        else:
            interpolations.append(x_1-idx*np.random.normal(delta, 0.1*delta))
    return interpolations

In [ ]:
def gen_signal():
    segment_length = [int(np.random.uniform(10, 30)) for _ in range(N_SEGMENTS)]
    segment_mean = [int(np.random.uniform(0, 20)) for _ in range(N_SEGMENTS)]
    random.shuffle(segment_mean)
    segment_list = [np.random.normal(segment_mean[idx], 1, segment_length[idx]) for idx in range(N_SEGMENTS)]

    signal = []
    cps = []
    for idx in range(N_SEGMENTS-1):
        signal.extend(segment_list[idx])
        prev_tail = segment_list[idx][-1]
        post_head = segment_list[idx+1][0]
        interpolations = gen_interpolation(prev_tail, post_head)
        if segment_mean[idx] != segment_mean[idx]+1:
            cps.append(len(signal)+len(interpolations)//2)
            signal.extend(interpolations)
    signal.extend(segment_list[-1])

    cp_one_hot = [0]*len(signal)
    for cp in cps:
        cp_one_hot[cp] = 1
    return signal, cp_one_hot

In [ ]:
def plot_signal(signal, cp_one_hot):
    fig, ax = plt.subplots()
    ax.plot(range(len(signal)), signal)
    for cp in np.where(np.array(cp_one_hot) == 1)[0]:
        ax.axvline(x=cp, color="red", linestyle="--")
    return fig, ax

In [ ]:
for idx in range(10):
    signal, cp_one_hot = gen_signal()
    fig, ax = plot_signal(signal, cp_one_hot)
    if not os.path.exists(IMAGE_ROOT):
        os.makedirs(IMAGE_ROOT)
    if not os.path.exists(SIGNAL_ROOT):
        os.makedirs(SIGNAL_ROOT)
    fig.savefig(os.path.join(IMAGE_ROOT, "signal_"+str(idx+1)+".pdf"))
    df = pd.DataFrame({"signal": signal, "is_change_point": cp_one_hot})
    df.to_csv(os.path.join(SIGNAL_ROOT, "signal_"+str(idx+1)+".csv"), index=False)